In [1]:
# Utils
from math import atan2

def orientation(a, b, c):
    ob = (b[0]-a[0], b[1]-a[1])
    oc = (c[0]-a[0], c[1]-a[1])
    det = ob[0]*oc[1]-ob[1]*oc[0]
    return -1 if det<0 else 1 if det>0 else 0

# Checks if the point b its on the segment ac
def onSegment(a,b,c):
    on_x_proyection = b[0]<=max(a[0],c[0]) and b[0]>= min(a[0],c[0])
    # print(f"pr_x: {on_x_proyection}")
    on_y_proyection = b[1]<=max(a[1],c[1]) and b[1]>= min(a[1],c[1])
    # print(f"pr_y: {on_y_proyection}")
    if(on_x_proyection and on_y_proyection): return True
    return False

def doIntersect(a,b,c,d):
    # Orientations between segment and the other points
    o_c_segment_ab = orientation(a,b,c)
    o_d_segment_ab = orientation(a,b,d)
    o_a_segment_cd = orientation(c,d,a)
    o_b_segment_cd = orientation(c,d,b)

    # Case 1
    if(o_c_segment_ab != o_d_segment_ab and o_a_segment_cd != o_b_segment_cd):
        return True
    
    # Special cases when they are colinear
    # print("Case 1")
    if(o_c_segment_ab == 0 and onSegment(a,c,b)): return True
    # print("Case 2")
    if(o_d_segment_ab == 0 and onSegment(a,d,b)): return True
    # print("Case 3")
    if(o_a_segment_cd == 0 and onSegment(c,a,d)): return True
    # print("Case 4")
    if(o_b_segment_cd == 0 and onSegment(c,b,d)): return True

    return False

def get_convex_hull(p):
    n = len(p)
    if n<3: return
    hull = []
    s_y = min(p, key=lambda x: x[1])
    p.sort(key=lambda x: (atan2(x[1]-s_y[1], x[0]-s_y[0]), -x[1], x[0]))
    for point in p:
        while len(hull)>=2 and orientation(hull[-2], hull[-1], point) <= 0:
            hull.pop()
        hull.append(point)
    return hull

# Finds the point of intersection between two segments
def get_intersection_point(s1,s2):
    x1, y1 = s1[0]
    x2, y2 = s1[1]
    x3, y3 = s2[0]
    x4, y4 = s2[1]
    # print(f"({x1},{y1})-({x2},{y2})")
    # print(f"({x3},{y3})-({x4},{y4})")
    # print("----")
    # Denominator
    den = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
    # print(f"den: {den}")
    if den == 0: return None
    # Numerators
    t = (x1-x3)*(y3-y4) - (y1-y3)*(x3-x4)
    u = (x1-x2)*(y1-y3) - (y1-y2)*(x1-x3)
    # print(f"t: {t}")
    # print(f"u: {u}")
    # print("----")
    t = t/den
    u = -u/den
    # print(f"t: {t}")
    # print(f"u: {u}")
    if t>=0 and t<=1 and u>=0 and u<=1:
        x = x1 + t*(x2-x1)
        y = y1 + t*(y2-y1)
        return (x,y)
    return None

# Sweep line for finding the intersection of segments

In [2]:
class Node:
    def __init__(self, key):
        self.key = key
        self.height = 1
        self.left = None
        self.right = None

class AVLTree:
    def get_height(self, root):
        if not root:
            return 0
        return root.height

    def get_balance(self, root):
        if not root:
            return 0
        return self.get_height(root.left) - self.get_height(root.right)

    def get_min_value_node(self, root):
        if root is None or root.left is None:
            return root
        return self.get_min_value_node(root.left)

    def preOrder(self, root):
        if not root:
            return
        # print("{0} ".format(root.key), end="")
        self.preOrder(root.left)
        self.preOrder(root.right)

    def insert(self, root, key):
        if not root:
            return Node(key)
        elif key < root.key:
            root.left = self.insert(root.left, key)
        else:
            root.right = self.insert(root.right, key)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance(root)

        if balance > 1 and key < root.left.key:
            return self.right_rotate(root)

        if balance < -1 and key > root.right.key:
            return self.left_rotate(root)

        if balance > 1 and key > root.left.key:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        if balance < -1 and key < root.right.key:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def left_rotate(self, z):
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y

    def right_rotate(self, y):
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))
        return x

myTree = AVLTree()
root = None
nums = [9, 5, 10, 0, 6, 11, -1, 1, 2]
for num in nums:
    root = myTree.insert(root, num)
myTree.preOrder(root)


9 1 0 -1 5 2 6 10 11 

In [3]:
# sweep line algorithm for finding the intersection of segments
def sweep_line(segments):
    # Create a list of events
    events = []
    Q = AVLTree()
    for segment in segments:
        events.append((segment[0][0], segment[0][1], 0, segment))
        events.append((segment[1][0], segment[1][1], 1, segment))
    events.sort()
    for event in events:
        x, y, t, segment = event
        if t == 0:
            Q.insert(segment)
            # print(f"Inserting {segment}")
            # print("Q:")
            # Q.preOrder(Q.root)
            # print("----")
            # print("----")
        else:
            Q.delete(segment)
            # print(f"Deleting {segment}")
            # print("Q:")
            # Q.preOrder(Q.root)
            # print("----")
            # print("----")
        # print("----")
    return None

segments = [((1,1),(3,3)),((2,2),(4,4)),((5,5),(7,7)),((6,6),(8,8))]
sweep_line(segments)

TypeError: AVLTree.insert() missing 1 required positional argument: 'key'